# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [ ]:
%help

####  Run this cell to set up and start your interactive session.


In [1]:
%idle_timeout 2880
%glue_version 3.0
%worker_type G.1X
%number_of_workers 2

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Current idle_timeout is 2880 minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 3.0
Previous worker type: G.1X
Setting new worker type to: G.1X
Previous number of workers: 2
Setting new number of workers to: 2
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 2
Session ID: 8cf04039-a925-439f-a4c9-8ee31e84ee82
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.38.1
--enable-glue-datacatalog true
Waiting for session 8cf04039-a925-439f-a4c9-8ee31e84ee82 to get into ready status...
Session 8cf04039-a925-439f-a4c9-8ee31e84ee82 has been created.



In [2]:
bggdf = glueContext.create_dynamic_frame.from_catalog(database="bgg-database", table_name="bgg_raw_xml_data_test")

In [3]:
bggdf.printSchema()

root
|-- _id: int
|-- _type: string
|-- description: string
|-- image: string
|-- link: array
|    |-- element: struct
|    |    |-- _VALUE: null
|    |    |-- _id: int
|    |    |-- _inbound: boolean
|    |    |-- _type: string
|    |    |-- _value: string
|-- maxplayers: struct
|    |-- _VALUE: null
|    |-- _value: int
|-- maxplaytime: struct
|    |-- _VALUE: null
|    |-- _value: int
|-- minage: struct
|    |-- _VALUE: null
|    |-- _value: int
|-- minplayers: struct
|    |-- _VALUE: null
|    |-- _value: int
|-- minplaytime: struct
|    |-- _VALUE: null
|    |-- _value: int
|-- name: choice
|    |-- array
|    |    |-- element: struct
|    |    |    |-- _VALUE: null
|    |    |    |-- _sortindex: int
|    |    |    |-- _type: string
|    |    |    |-- _value: string
|    |-- struct
|    |    |-- _VALUE: null
|    |    |-- _sortindex: int
|    |    |-- _type: string
|    |    |-- _value: string
|-- playingtime: struct
|    |-- _VALUE: null
|    |-- _value: int
|-- poll: array
|    

In [4]:
bgg_relationalize = bggdf.relationalize("root", "s3://bgg-raw-xml-data-test/glue_relationalize/")
bgg_relationalize.keys()

dict_keys(['root', 'root_poll.val.results.val.result', 'root_poll.val.results', 'root_link', 'root_statistics.ratings.ranks.rank', 'root_name.array', 'root_poll'])


In [6]:
bggflatdf = bgg_relationalize.select("root")
bggflatdf.printSchema()

root
|-- _id: int
|-- _type: string
|-- description: string
|-- image: string
|-- link: long
|-- maxplayers._VALUE: null
|-- maxplayers._value: int
|-- maxplaytime._VALUE: null
|-- maxplaytime._value: int
|-- minage._VALUE: null
|-- minage._value: int
|-- minplayers._VALUE: null
|-- minplayers._value: int
|-- minplaytime._VALUE: null
|-- minplaytime._value: int
|-- name.array: long
|-- name.struct._VALUE: null
|-- name.struct._sortindex: int
|-- name.struct._type: string
|-- name.struct._value: string
|-- playingtime._VALUE: null
|-- playingtime._value: int
|-- poll: long
|-- statistics._page: int
|-- statistics.ratings.average._VALUE: null
|-- statistics.ratings.average._value: double
|-- statistics.ratings.averageweight._VALUE: null
|-- statistics.ratings.averageweight._value.double: double
|-- statistics.ratings.averageweight._value.int: int
|-- statistics.ratings.bayesaverage._VALUE: null
|-- statistics.ratings.bayesaverage._value: double
|-- statistics.ratings.median._VALUE: null


In [13]:
bgg_relationalize.select("root").toDF().show()

+------+---------+--------------------+--------------------+----+-----------------+-----------------+------------------+------------------+-------------+-------------+-----------------+-----------------+------------------+------------------+----------+------------------+----------------------+-----------------+------------------+------------------+------------------+----+----------------+---------------------------------+---------------------------------+---------------------------------------+----------------------------------------------+-------------------------------------------+--------------------------------------+--------------------------------------+--------------------------------+--------------------------------+-------------------------------------+-------------------------------------+------------------------------------+------------------------------------+-------------------------------+-------------------------------+-----------------------------+----------------------

In [7]:
bgg_relationalize.select("root_poll.val.results.val.result").printSchema()

root
|-- id: long
|-- index: int
|-- poll.val.results.val.result.val._numvotes: int
|-- poll.val.results.val.result.val._value: string
|-- poll.val.results.val.result.val._level: int


In [14]:
bgg_relationalize.select("root_poll.val.results.val.result").toDF().show()

+---+-----+-----------------------------------------+--------------------------------------+--------------------------------------+
| id|index|poll.val.results.val.result.val._numvotes|poll.val.results.val.result.val._value|poll.val.results.val.result.val._level|
+---+-----+-----------------------------------------+--------------------------------------+--------------------------------------+
|  1|    0|                                        0|                                  Best|                                  null|
|  1|    1|                                        4|                           Recommended|                                  null|
|  1|    2|                                      267|                       Not Recommended|                                  null|
|  2|    0|                                       58|                                  Best|                                  null|
|  2|    1|                                      223|                       

In [8]:
bgg_relationalize.select("root_poll.val.results").printSchema()

root
|-- id: long
|-- index: int
|-- poll.val.results.val._numplayers: string
|-- poll.val.results.val.result: long


In [15]:
bgg_relationalize.select("root_poll.val.results").toDF().show()

+---+-----+--------------------------------+---------------------------+
| id|index|poll.val.results.val._numplayers|poll.val.results.val.result|
+---+-----+--------------------------------+---------------------------+
|  1|    0|                               1|                          1|
|  1|    1|                               2|                          2|
|  1|    2|                               3|                          3|
|  1|    3|                               4|                          4|
|  1|    4|                              4+|                          5|
|  2|    0|                                |                          6|
|  3|    0|                                |                          7|
|  4|    0|                               1|                          8|
|  4|    1|                               2|                          9|
|  4|    2|                               3|                         10|
|  4|    3|                               4|       

In [9]:
bgg_relationalize.select("root_link").printSchema()

root
|-- id: long
|-- index: int
|-- link.val._id: int
|-- link.val._type: string
|-- link.val._value: string
|-- link.val._inbound: boolean


In [16]:
bgg_relationalize.select("root_link").toDF().show()

+---+-----+------------+-----------------+--------------------+-----------------+
| id|index|link.val._id|   link.val._type|     link.val._value|link.val._inbound|
+---+-----+------------+-----------------+--------------------+-----------------+
|  1|    0|        1021|boardgamecategory|            Economic|             null|
|  1|    1|        1088|boardgamecategory|Industry / Manufa...|             null|
|  1|    2|        2710|boardgamecategory|     Post-Napoleonic|             null|
|  1|    3|        1011|boardgamecategory|      Transportation|             null|
|  1|    4|        2040|boardgamemechanic|     Hand Management|             null|
|  1|    5|        2902|boardgamemechanic|              Income|             null|
|  1|    6|        2904|boardgamemechanic|               Loans|             null|
|  1|    7|        2900|boardgamemechanic|              Market|             null|
|  1|    8|        2081|boardgamemechanic|Network and Route...|             null|
|  1|    9|     

In [10]:
bgg_relationalize.select("root_statistics.ratings.ranks.rank").printSchema()

root
|-- id: long
|-- index: int
|-- statistics.ratings.ranks.rank.val._bayesaverage: double
|-- statistics.ratings.ranks.rank.val._friendlyname: string
|-- statistics.ratings.ranks.rank.val._id: int
|-- statistics.ratings.ranks.rank.val._name: string
|-- statistics.ratings.ranks.rank.val._type: string
|-- statistics.ratings.ranks.rank.val._value: int


In [17]:
bgg_relationalize.select("root_statistics.ratings.ranks.rank").toDF().show()

+---+-----+-----------------------------------------------+-----------------------------------------------+-------------------------------------+---------------------------------------+---------------------------------------+----------------------------------------+
| id|index|statistics.ratings.ranks.rank.val._bayesaverage|statistics.ratings.ranks.rank.val._friendlyname|statistics.ratings.ranks.rank.val._id|statistics.ratings.ranks.rank.val._name|statistics.ratings.ranks.rank.val._type|statistics.ratings.ranks.rank.val._value|
+---+-----+-----------------------------------------------+-----------------------------------------------+-------------------------------------+---------------------------------------+---------------------------------------+----------------------------------------+
|  1|    0|                                        7.96473|                                Board Game Rank|                                    1|                              boardgame|              

In [11]:
bgg_relationalize.select("root_name.array").printSchema()

root
|-- id: long
|-- index: int
|-- name.array.val._sortindex: int
|-- name.array.val._type: string
|-- name.array.val._value: string


In [18]:
bgg_relationalize.select("root_name.array").toDF().show()

+---+-----+-------------------------+--------------------+--------------------------+
| id|index|name.array.val._sortindex|name.array.val._type|     name.array.val._value|
+---+-----+-------------------------+--------------------+--------------------------+
|  1|    0|                        1|             primary|         Brass: Lancashire|
|  1|    1|                        1|           alternate|                     Brass|
|  1|    2|                        1|           alternate|              Brass Deluxe|
|  1|    3|                        1|           alternate|           Brass. Ланкашир|
|  1|    4|                        1|           alternate|         Brass: Lancashire|
|  1|    5|                        1|           alternate|      Kohle: Mit Vollda...|
|  1|    6|                        1|           alternate|      ブラス：ランカシャー|
|  1|    7|                        1|           alternate|                  工業革命|
|  1|    8|                        1|           alternate|          

In [12]:
bgg_relationalize.select("root_poll").printSchema()

root
|-- id: long
|-- index: int
|-- poll.val._name: string
|-- poll.val._title: string
|-- poll.val._totalvotes: int
|-- poll.val.results: long


In [19]:
bgg_relationalize.select("root_poll").toDF().show()

+---+-----+--------------------+--------------------+--------------------+----------------+
| id|index|      poll.val._name|     poll.val._title|poll.val._totalvotes|poll.val.results|
+---+-----+--------------------+--------------------+--------------------+----------------+
|  1|    0|suggested_numplayers|User Suggested Nu...|                 577|               1|
|  1|    1| suggested_playerage|User Suggested Pl...|                  95|               2|
|  1|    2| language_dependence| Language Dependence|                 100|               3|
|  2|    0|suggested_numplayers|User Suggested Nu...|                 688|               4|
|  2|    1| suggested_playerage|User Suggested Pl...|                 116|               5|
|  2|    2| language_dependence| Language Dependence|                  22|               6|
|  3|    0|suggested_numplayers|User Suggested Nu...|                 387|               7|
|  3|    1| suggested_playerage|User Suggested Pl...|                  80|      